In [1]:
import sklearn
print(sklearn.__version__)


1.3.2


In [ ]:

features_train, target_train, features_test = prepare_features_and_target(train_df, test_df, config)

if config["use_feature_gen"]:
    features_train = add_new_features(features_train)
    features_test  = add_new_features(features_test)

cat_cols = ['road_type', 'lighting', 'weather', 'time_of_day',
            'RoadType_Time', 'Lighting_Weather', 'Accident_Bins']                # 类别交互项,特征生成


# 1. 在训练集上拟合
ohe = fit_ohe(features_train, cat_cols)

# 2. 分别编码 train/test，保证一致
features_train = transform_ohe(features_train, ohe, cat_cols)
features_test  = transform_ohe(features_test,  ohe, cat_cols)



# 对数据降维
if config["use_pca"]:
    # 1. 在训练集上拟合
    svd = fit_svd(features_train, n_components = config["pca_components"], random_state=42)

    # 2. 分别对 train/test transform
    features_train_reduced = transform_svd(features_train, svd)
    features_test_reduced  = transform_svd(features_test, svd)

    


    
    shape_before_train = features_train.shape
    shape_before_test = features_test.shape

    features_train = pd.concat([features_train, features_train_reduced], axis=1)
    features_test = pd.concat([features_test, features_test_reduced], axis=1)

    shape_after_train = features_train.shape
    shape_after_test = features_test.shape

    print(f"Train: {shape_before_train[0]} × {shape_before_train[1]}  -->  {shape_after_train[0]} × {shape_after_train[1]}   新增 {shape_after_train[1] - shape_before_train[1]} 列")
    print(f"Test : {shape_before_test[0]} × {shape_before_test[1]}  -->  {shape_after_test[0]} × {shape_after_test[1]}   新增 {shape_after_test[1] - shape_before_test[1]} 列")


X = features_train
y = target_train
X_test = features_test


results = run_kfold_xgb(X, y, X_test, config, DIRS, K_FOLDS = 10, verbose = 0)
